In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# # Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install torchsummary

In [3]:
import torch
from torch.utils.data import Dataset, Subset, SubsetRandomSampler, SequentialSampler
from torch.utils.data.dataset import TensorDataset
from torch.distributions.multivariate_normal import MultivariateNormal
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx
torch.manual_seed(0)

import networkx as nx # for visualizing graphs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import pdb
from torchsummary import summary
import json
import pickle
import os

from functions.load_data import MarielDataset, edges
from functions.functions import *
from functions.modules import *
from functions.seq_autoencoder import *

# Test

In [ ]:
# ! ls -latrh ./logs/overfit

In [5]:
folder = "./logs/pred_seq10"
dataloader_test = torch.load(os.path.join(folder,"dataloader_test.pth")) ## override with my longer dataset loader
dataloader_train = torch.load(os.path.join(folder,"dataloader_train.pth"))
checkpoint_path = os.path.join(folder,"best_weights.pth")
args_file = os.path.join(folder, 'args.pkl')
args = pickle.load(open(args_file, "rb" ))['args']
checkpoint_loaded = False 
print(args)

# Load these if training actually completed:
if os.path.exists(os.path.join(folder,"losses.json")):
    print("Loaded losses")
    dict = json.load(open(os.path.join(folder,"losses.json")))
    train_losses = dict['train_losses']
    val_losses = dict['val_losses']
    
    fig, ax = plt.subplots(figsize=(8,6))
    ax.plot(np.arange(len(train_losses)), train_losses, label="Training")
    ax.plot(np.arange(len(val_losses)), val_losses, label="Validation")
    ax.set_xlabel("Epoch", fontsize=16)
    ax.set_ylabel("NLL Loss", fontsize=16)
    # ax.set_yscale("log")
#     ax.set_ylim(-0.05,1)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    ax.legend(fontsize=14)

Namespace(batch_limit=0, batch_size=6, dynamic_graph=False, edge_embedding_dim=4, epochs=50, hidden_size=256, lr=0.0005, name='pred_seq10', no_cuda=False, no_overlap=False, node_embedding_dim=36, num_layers=1, pred_to_reco_ratio=1.0, predicted_timesteps=2, reduced_joints=False, seq_len=10, shuffle=False, skip_connection=False)


In [6]:
if torch.cuda.is_available() and args.no_cuda == False:
    device = torch.device('cuda')
else:
    device = 'cpu'

model = NRI(device=device,
            node_features=args.seq_len*6, 
            edge_features=args.seq_len, 
            hidden_size=args.hidden_size, 
            node_embedding_dim=args.node_embedding_dim,
            edge_embedding_dim=args.edge_embedding_dim,
            skip_connection=args.skip_connection,
            dynamic_graph=args.dynamic_graph,
            seq_len=args.seq_len,
            predicted_timesteps=args.predicted_timesteps,
           )

optimizer = torch.optim.Adam(list(model.parameters()), lr=args.lr, weight_decay=5e-4)

print("Using {}".format(device))
model = model.to(device)
print(model)
print("Total trainable parameters: {:,}".format(count_parameters(model)))

Using cuda
NRI(
  (encoder): NRIEncoder(
    (node_embedding_eqn_5): Sequential(
      (0): Linear(in_features=60, out_features=36, bias=True)
      (1): ReLU()
    )
    (mlp_eqn_6): Sequential(
      (0): Linear(in_features=72, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=36, bias=True)
    )
    (mlp_eqn_7): Sequential(
      (0): Linear(in_features=36, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=36, bias=True)
    )
    (mlp_eqn_8): Sequential(
      (0): Linear(in_features=72, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=4, bias=True)
    )
    (graph_conv): NRIGraphConv(36, 36)
  )
  (decoder): NRIDecoder(
    (encoder): NRIEncoder(
      (node_embedding_eqn_5): Sequential(
        (0): Linear(in_features=60, out_features=36, bias=True)
        (1): ReLU()
      )
      (mlp_eqn_6): Sequential(
        (0): Linear(in_features=72, o

In [7]:
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss_checkpoint = checkpoint['loss']
checkpoint_loaded = True
n_joints = 53

In [8]:
loss_checkpoint

22.878156237897336

In [9]:
def test(batch_limit=0):
    mse_loss = torch.nn.MSELoss(reduction='mean')
    prediction_to_reconstruction_loss_ratio = 1 # CHANGE THIS TO ARGS
    total_test_loss = 0
    n_batches = 0
    actuals = []
    preds = []
    model.eval()
    for batch in tqdm(dataloader_test, desc="Test batches"):
        batch = batch.to(device)
        print(batch.x.size())

        ### CALCULATE MODEL OUTPUTS
        output, edges = model(batch)
        
        ### SAVE FOR ANIMATIONS
        actuals.append(batch.x.detach().cpu().numpy())
        preds.append(output.detach().cpu().numpy())

        ### CALCULATE LOSS
        test_loss = mse_loss(batch.x.to(device), output) # compare first seq_len timesteps

        ### ADD LOSSES TO TOTALS
        total_test_loss += test_loss.item()
            
        ### OPTIONAL -- STOP TESTING EARLY
        n_batches += 1
        if (batch_limit > 0) and (n_batches >= batch_limit): break # temporary -- for stopping training early

    ### CALCULATE AVERAGE LOSSES PER EPOCH   
    average_test_loss = total_test_loss / n_batches
    print("Loss = {:,.8f}".format(average_test_loss))
    return actuals, preds

In [ ]:
actuals, preds = test(batch_limit = 1)

In [ ]:
### ANIMATE A SINGLE BATCH ONLY 
batch_number = 0
truth_sequences= []
predicted_sequences = []

for seq_number in np.arange(args.batch_size):
    actual = actuals[batch_number][seq_number*n_joints:seq_number*n_joints+n_joints].reshape((n_joints,args.seq_len,6))[:,:,:3] # take the first 3 dimensions for positions, not velocities
    pred = preds[batch_number][seq_number*n_joints:seq_number*n_joints+n_joints].reshape((n_joints,args.seq_len,6))[:,:,:3]
    actual = np.transpose(actual, [1,0,2])
    pred = np.transpose(pred, [1,0,2])
    truth_sequences.append(actual)
    predicted_sequences.append(pred)
    
truth_sequences = np.asarray(truth_sequences).reshape((args.batch_size*args.seq_len, n_joints, 3))
predicted_sequences = np.asarray(predicted_sequences).reshape((args.batch_size*args.seq_len, n_joints, 3))

start_index = 0
# timesteps = args.seq_len*args.batch_size
timesteps = args.seq_len
animation = animate_stick(truth_sequences[start_index:start_index+timesteps,:,:], 
                          ghost=predicted_sequences[start_index:start_index+timesteps,:,:], 
                          ghost_shift=0.,
                          ax_lims = (-0.5,0.5),
                          skeleton=True,
                          skeleton_alpha=1,
                          figsize=(10,8), cmap='inferno')
HTML(animation.to_html5_video())

In [ ]:
### ANIMATE MULTIPLE BATCHES AT ONCE 
batch_limit = 5
truth_sequences= []
predicted_sequences = []

for batch_number in np.arange(batch_limit):
    for seq_number in np.arange(args.batch_size):
        actual = actuals[batch_number][seq_number*n_joints:seq_number*n_joints+n_joints].reshape((n_joints,args.seq_len,6))[:,:,:3] # take the first 3 dimensions for positions, not velocities
        pred = preds[batch_number][seq_number*n_joints:seq_number*n_joints+n_joints].reshape((n_joints,args.seq_len,6))[:,:,:3]
        actual = np.transpose(actual, [1,0,2])
        pred = np.transpose(pred, [1,0,2])
        truth_sequences.append(actual)
        predicted_sequences.append(pred)
    
truth_sequences = np.asarray(truth_sequences).reshape((batch_limit*args.batch_size*args.seq_len, n_joints, 3))
predicted_sequences = np.asarray(predicted_sequences).reshape((batch_limit*args.batch_size*args.seq_len, n_joints, 3))

animation = animate_stick(truth_sequences, 
                          ghost=predicted_sequences, 
                          ghost_shift=0.4,
                          ax_lims = (-0.5,0.5),
                          figsize=(10,8), cmap='inferno')
HTML(animation.to_html5_video())

# Test w/ predictions

In [10]:
data = MarielDataset(seq_len=100, reduced_joints=False, predicted_timesteps=2, no_overlap=True)
test_indices = np.arange(int(0.85*len(data)), len(data)) # last 15% on test
test = torch.utils.data.Subset(data, test_indices)
dataloader_test_long = DataLoader(test, batch_size=1, shuffle=False, drop_last=True)

Original numpy dataset contains 38,309 timesteps of 53 joints with 3 dimensions each.

Generating non-overlapping sequences...
Using (x,y)-centering...
Using all joints...


In [11]:
def test_pred(batch_limit=0):
    mse_loss = torch.nn.MSELoss(reduction='mean')
    prediction_to_reconstruction_loss_ratio = 1 # CHANGE THIS TO ARGS
    total_test_loss = 0
    n_batches = 0
    actuals_all = []
    preds_all = []
    preds_all_for_animation = []
    model.eval()
    for batch in tqdm(dataloader_test_long, desc="Test batches"):
        actuals = []
        preds = []
        preds_for_animation = []
        batch = batch.to(device)
#         n_predictions = int(batch.x.size(1)/(6*args.seq_len))
#         print("This batch has {} full sequences in it.".format(n_predictions))

        n_predictions=3
        for i in range(n_predictions):
            if i == 0 :
                x = batch.x[:,:args.seq_len*6]
                input_seq = Data(x=x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
            elif i == 1: 
                preds_for_animation.append(preds[i-1]) # start out the animation of predictions with the real steps + first predictions
                x_real = batch.x[:,:(args.seq_len-args.predicted_timesteps)*6]
                x_pred = torch.tensor(preds[i-1][:,-args.predicted_timesteps*6:]).to(device) # add on the previous sequence's predictions
                x = torch.cat([x_real, x_pred], axis=1)
                input_seq = Data(x=x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
                preds_for_animation.append(x_pred.detach().cpu().numpy())
            elif args.seq_len > i*args.predicted_timesteps:
                    x_real = batch.x[:,:(args.seq_len-i*args.predicted_timesteps)*6]
                    new_pred = torch.tensor(preds[i-1][:,-args.predicted_timesteps*6:]).to(device)
                    x_pred = torch.cat([x_pred, new_pred], axis=1) # add on the previous sequence's predictions
                    x = torch.cat([x_real, x_pred], axis=1)
                    input_seq = Data(x=x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
                    preds_for_animation.append(new_pred.detach().cpu().numpy())
            elif args.seq_len == i*args.predicted_timesteps:
                x = torch.tensor(preds[i-1]).to(device) # now this is the right size to feed into the model
                input_seq = Data(x=x, edge_index=batch.edge_index, edge_attr=batch.edge_attr)
                new_pred = torch.tensor(preds[i-1][:,-args.predicted_timesteps*6:]).to(device)
                preds_for_animation.append(new_pred.detach().cpu().numpy())
            else:
                continue
                
            ### CALCULATE MODEL OUTPUTS
            output, edges = model(input_seq)

            ### SAVE FOR ANIMATIONS  
            preds.append(output.detach().cpu().numpy())
            
    
        ### STACK
        actuals_all.append(batch.x.detach().cpu().numpy())
        preds_all.append(np.hstack(preds))
        preds_all_for_animation.append(np.hstack(preds_for_animation))
        
        ### CALCULATE LOSS
#         test_loss = mse_loss(inputs, outputs) # compare first seq_len timesteps

        ### ADD LOSSES TO TOTALS
#         total_test_loss += test_loss.item()
            
        ### OPTIONAL -- STOP TESTING EARLY
        n_batches += 1
        if (batch_limit > 0) and (n_batches >= batch_limit): break # temporary -- for stopping training early

    ### CALCULATE AVERAGE LOSSES PER EPOCH   
#     average_test_loss = total_test_loss / n_batches
#     print("Loss = {:,.8f}".format(average_test_loss))
    return actuals_all, preds_all, preds_all_for_animation, n_predictions, n_batches

In [12]:
actuals, preds, preds_animation, n_predictions, n_batches = test_pred(batch_limit = 1)

Test batches:   0%|          | 0/58 [00:00<?, ?it/s]


In [13]:
pred_steps = int(preds_animation[0].shape[1]/6)
print(pred_steps)

14


In [18]:
### ANIMATE A SINGLE BATCH ONLY 
batch_number = 0
truth_sequences= []
predicted_sequences = []

actual = actuals[batch_number].reshape((n_joints,100,6))[:,:,:3] # take the first 3 dimensions for positions, not velocities
pred = preds_animation[batch_number].reshape((n_joints,pred_steps,6))[:,:,:3]
actual = np.transpose(actual, [1,0,2])
pred = np.transpose(pred, [1,0,2])
truth_sequences.append(actual)
predicted_sequences.append(pred)
    
truth_sequences = np.asarray(truth_sequences).reshape((100, n_joints, 3))
predicted_sequences = np.asarray(predicted_sequences).reshape((pred_steps, n_joints, 3))

# timesteps = args.seq_len*args.batch_size
timesteps = pred_steps
animation = animate_stick(truth_sequences[:timesteps,:,:], 
                          ghost=predicted_sequences[:timesteps,:,:], 
                          ghost_shift=0.,
                          ax_lims = (-0.5,0.5),
                          skeleton=True,
                          skeleton_alpha=1,
                          figsize=(10,8), cmap='inferno')
HTML(animation.to_html5_video())

Skeleton lines: 100%|██████████| 74/74 [00:00<00:00, 1028.30it/s]


In [17]:
31800/53/6

100.0

# Misc

In [ ]:
# sampling_recurrent = [0.0969,0.0103,0.0100,0.0067,0.0066,0.0063,0.0056,0.0051,0.0042,0.0041,0.0048,0.0027,0.0016,0.0037,0.0038,0.0031,0.0036,0.0017,0.0016,0.0041,0.0089,]
# nosampling_recurrent = [0.6768,0.3889,0.0499,0.0085,0.0081,0.0079,0.0080,0.4788,0.0412,0.1520,0.0554,0.0090,0.0088,0.0088,0.0088,223.3473,0.0089,0.0186,0.0092,0.0089,0.0093,]
# sampling_norecurrent = [6,419,004.4403,1,180,173.9217,12,472.5292,5,309.1666,3,601.1410,23,310.8198,10,171,718.4292,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,962.9072,]
# nosampling_norecurrent = [16,822,800.1428,651,681.9966,187,729.5295,211,615.4727,334,911.2555,183,381.5607,28,994.0494,28,142.1309,28,039.8979,27,708.8960,26,805.5452,23,027.8847,10,154.9895,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,0.0089,]
# val_losses_moreparams = [1.2585,0.2413,0.2063,0.3304,0.1753,0.1665,0.1516,0.2043,0.1444,32.0709,0.1563,15.5246,0.7867,1.8139,2.1597,0.1700,0.1611,0.1606,0.1605,0.1671,0.1589,0.7795,0.1553,0.1550,0.1546,0.1544,0.1546,0.1546,0.1522,0.1533,0.1545,0.1509,0.1521,0.1538,0.1538,0.1467,0.1402,0.1433,0.1413,0.1402,0.1425,0.1419,0.1372,0.1359,0.1319,0.1331,0.1354,0.1371,0.1360,0.1283,0.1283,0.1277,0.1277,0.1271,0.1353,0.1353,0.1390,0.1249,0.0118,0.0040,0.1584,0.1568,0.1558,0.1556,0.1556,0.1554,0.1536,0.1535,0.1533,0.1523,0.1419,0.1338,]

In [ ]:
# fig, ax = plt.subplots(figsize=(8,6))
# # ax.plot(np.arange(len(val_losses)), val_losses, label="Graph VAE (250k params)")
# # ax.plot(np.arange(len(val_losses_moreparams)), val_losses_moreparams, label="Graph VAE (1.8M params)")
# # ax.plot(np.arange(len(sampling_recurrent)), sampling_recurrent, label="Sampling, Recurrent")
# # ax.plot(np.arange(len(nosampling_recurrent)), nosampling_recurrent, label="No Sampling, Recurrent")
# ax.plot(np.arange(len(sampling_norecurrent)), sampling_norecurrent, label="Sampling, No Recurrent")
# ax.plot(np.arange(len(nosampling_norecurrent)), nosampling_norecurrent, label="No Sampling, No Recurrent")
# ax.set_xlabel("Epoch", fontsize=16)
# ax.set_ylabel("Validation Reco Loss", fontsize=16)
# # ax.set_yscale("log")
# # ax.set_ylim(0,0.1)
# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)
# ax.legend(fontsize=14)

### Up next:
- Weight the prediction loss to ensure that the most immediate steps are more important to reconstruct than far future steps (like 1/2^t or something)

### Done
- ~~Predict 50 + k timesteps w/ separate MSE losses~~
- ~~Look at VAE outputs!~~
- ~~Look at NRI outputs!~~
- ~~The Gaussian negative log likelihood loss functions will only make sense when the output of the decoder is mu (eq'n 16 & 17)~~
- ~~Try to overfit the input data (loss = 0) by setting prediction weight = 0~~